<div style="text-align: right">INFO 6105 - Data Science Eng Methods and Tools</div>
<div style="text-align: right">Assignment 4</div>
<div style="text-align: right">Vignesh Kumar Baskar [002196442]</div>

**Movies with Python**

**Team Submission - Software**

*Team 4_2*


*Adding borders to the list of images*

In [ ]:
from PIL import Image as pili, ImageOps as piliops

#path_in = r"ENG/Frames/"
#path_out = r"ENG/Borders/"

list_im = os.listdir(path_in)

for files in os.listdir(path_in):
    new_path = path_in+files
    new_path_out = path_out+files
    piliops.expand(pili.open(new_path), border=(170,170),fill='blue').save(new_path_out)
    
print("Success!!")

*Importing Subtiles from the Excell sheet*

*Accessing those using Pandas library*

In [ ]:
import pandas as pd

subs = []
df = pd.read_excel('Scripts/Script.xlsx', sheet_name='Sheet1', usecols='B')
df = df['Subtitle-ENG'].str.replace("^\['|'\]$","")
for i in df.values:
    subs.append(i)

*Adding the subtitle to the bordered images and saved in the FrameWithSubs Folder*

In [ ]:
from PIL import Image as pili, ImageDraw as pild, ImageFont as pilf
from PIL import ImageOps

#path_in = r"ENG/Borders/"
#path_out = r"ENG/FrameWithSubs/"

list_im = os.listdir(path_in)
list_im.sort()
index = 0


for files in os.listdir(path_in):
    
    new_path = path_in+files
    image = pili.open(new_path)
    resize = image.resize((500,500))
    border = ImageOps.expand(resize, border=(8,8), fill=(600,400,200))
    
    TINT_COLOR = (0, 0, 0)  # Black
    TRANSPARENCY = .40      # Degree of transparency, 0-100%
    OPACITY = int(255 * TRANSPARENCY)
        
    img = border.convert('RGBA')
    overlay = pili.new('RGBA', img.size, TINT_COLOR+(0,))
    draw = pild.Draw(overlay)
    font = pilf.truetype("arial.ttf", 30)
    text = subs[index]
    w, h = font.getsize(text)
    num_lines = len(text.split('\n'))
    x, y = 10, img.height - (num_lines-0.1*num_lines)*h
    draw.rectangle((x, y, x + img.width - 200, y + (num_lines-0.1*num_lines)*h), fill=TINT_COLOR+(OPACITY,))
    draw.text((x, y), text, fill=(209, 239, 8), font=font)

    # Alpha composite these two images together to obtain the desired result.
    img = pili.alpha_composite(img, overlay)
    
    img = img.convert("RGB")   # Remove alpha for saving in jpg format.
    new_path_out = path_out+files
    img.save(new_path_out)
    index+= 1
    
print("Success!!")

*Stacking those images horizontally*

In [ ]:
import os
import numpy as np
import PIL
from os.path import isfile, join
#path_in = r"ENG/FrameWithSubs/"
#path_out = r"ENG/Horizontal/"
files = os.listdir(path_in)
files.sort()

pages = 1
while 0 < len(files):
    lines = 0
    print('PAGE '+str(pages))
    num_cols = old_num_cols = 0
    while lines < 6 and 0 < len(files):
        lines += 1
        while num_cols == old_num_cols:
            num_cols = np.random.randint(3, 5)
        old_num_cols = num_cols
        files_one_line = files[:num_cols]
        print(files_one_line)
        imgs_one_line = [ PIL.Image.open(path_in+i) for i in files_one_line ]
        min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs_one_line])[0][1]
        imgs_comb = np.hstack( (np.asarray(i.resize(min_shape)) for i in imgs_one_line ) )
        imgs_comb = PIL.Image.fromarray(imgs_comb)
        imgs_comb.save( path_out+str(pages)+str(lines)+'.jpg' )
        files = files[num_cols:]
    pages+=1

print("\nSuccess!!")

*Vertically stacking those horizontal images*

In [ ]:
import os
import numpy as np
import PIL
from os.path import isfile, join

#path_in = r"ENG/Horizontal/"
#path_out = r"ENG/Vertical/"

files = os.listdir(path_in)
files.sort()

pages = 1
while 0 < len(files):
    lines = 0
    print('page '+str(pages))
    num_cols = old_num_cols = 0
    while lines < 6 and 0 < len(files):
        lines += 1
        while num_cols == old_num_cols:
            num_cols = np.random.randint(3, 5)
        old_num_cols = num_cols
        files_one_line = files[:num_cols]
        print(files_one_line)
        imgs_one_line = [ PIL.Image.open(path_in+i) for i in files_one_line ]
        min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs_one_line])[0][1]
        imgs_comb = np.vstack( (np.asarray(i.resize(min_shape)) for i in imgs_one_line ) )
        imgs_comb = PIL.Image.fromarray(imgs_comb)
        imgs_comb.save( path_out+str(pages)+str(lines)+'.jpg' )
        files = files[num_cols:]
    pages+=1

print("\nSuccess!!")

*Adding cartoon comic mix to those frames*

Defining two functions: 

1. imgcompress_mem - For compressing and resizing the image

2. cartoonizebl_mem - For adding cartoonist feature to those images by using cv2 library

In [ ]:
import cv2 
def imgcompress_mem(path_in, k):
    img = cv2.imread(path_in, cv2.IMREAD_UNCHANGED)

    # set the ratio of resized image
    width = int((img.shape[1])/k)
    height = int((img.shape[0])/k)

    # resize the image by resize() function of openCV library
    return cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA)

In [ ]:
def cartoonizebl_mem(path_in, k, blur, line):
    
    imgc = imgcompress_mem(path_in, k)
    #imgc_pil = cv2.cvtColor(imgc, cv2.COLOR_BGR2RGB) # Converting BGR to RGB
    #display(Image.fromarray(imgc_pil))

    line_size = line
    blur_value = blur
    #imgc = cv2.imread(path_out, cv2.IMREAD_UNCHANGED)
    gray = cv2.cvtColor(imgc, cv2.COLOR_BGR2GRAY)
    gray_blur = cv2.medianBlur(gray, blur_value)
    bigedges = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, line_size, blur_value)
    bigedges_pil = cv2.cvtColor(bigedges, cv2.COLOR_BGR2RGB) # Converting BGR to RGB
    #display(Image.fromarray(bigedges_pil))

    return cv2.bitwise_and(imgc, imgc, mask=bigedges)

In [ ]:
import cv2 
from IPython.display import display
import numpy as np
from PIL import Image, ImageOps

#path_in = r"ENG/Vertical/"
files = os.listdir(path_in)
#path_out = r"ENG/Animation/"

for image in files:
    
    new_path = path_in+image
    cblimg = cartoonizebl_mem(new_path, 1.5, 5, 5)
    cv2.imwrite(path_out+image,cblimg)

*Step6: Converting these images to PDF file and save it in PDF folder*

In [ ]:
from PIL import Image
from fpdf import FPDF

#path = r"ENG/Animation/"
#cover = Image.open('ENG/Animation/11.jpg')
width, height = cover.size
pdf = FPDF(unit = "pt", format = [width, height])
    
files = os.listdir(path)

for image in files:
    new_path = path+image
    pdf.add_page()
    pdf.image(new_path, 0, 0, width, height)
    
pdf.output("ENG/PDF/ComicBook_ENG.pdf", "F")
print("Success!! Your Comic book PDF version has been saved.")